In [1]:
!apt install tesseract-ocr libtesseract-dev
!pip install -q -U google-generativeai chromadb pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libarchive-dev libleptonica-dev tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  libarchive-dev libleptonica-dev libtesseract-dev tesseract-ocr tesseract-ocr-eng
  tesseract-ocr-osd
0 upgraded, 6 newly installed, 0 to remove and 45 not upgraded.
Need to get 8,560 kB of archives.
After this operation, 31.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libarchive-dev amd64 3.6.0-1ubuntu1.1 [582 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libleptonica-dev amd64 1.82.0-3build1 [1,562 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libtesseract-dev amd64 4.1.1-2.1build1 [1,600 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:5 http://arc

In [2]:
import time
from tqdm import tqdm
import pathlib
import google.generativeai as genai
from google.generativeai import GenerativeModel
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
import pandas as pd
from PIL import Image
import pytesseract
from IPython.display import Markdown

In [3]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

# Gemini Final Exercise

You're an astronomy student who's very curious about the Apollo 11 missions,
and through your research, you've found a lot of different types of data (otherwise known as multimodal) from NASA's
public archive.

1. Text: You have the full final NASA report post-mission, spanning over 300
pages of incredibly informative content that details a summary of everything
that happened as well as conclusions that NASA researchers and engineers
came to. For the sake of this exercise, we've selected 3 particularly interesting pages, and converted them to images (you'll see soon why).

2. Video: You also have several clips of the famous Neil Armstrong and Buzz Aldrin footage as they
first stepped onto the moon, containing highlights of their moonwalks as well
as raising the American flag.

3. Audio: Finally, you have highlights from the audio recorded throughout the
mission, which provides insights into how communication between the astronauts
occurred as well as from the astronauts to mission control.

Now, you want to search through and summarize this information for your
upcoming research paper. Using your newfound skills from this course, you
can accomplish this using Gemini! In particular, we will build a Retrieval Augmented Generation (RAG) system that you can directly interact with.

## Data Preparation

Before we begin, ensure that you've uploaded the resources.zip folder and unzipped it using the following command:

In [4]:
!wget -O resources.zip "https://video.udacity-data.com/topher/2024/June/66744e79_resources/resources.zip"

--2024-07-20 08:24:06--  https://video.udacity-data.com/topher/2024/June/66744e79_resources/resources.zip
Resolving video.udacity-data.com (video.udacity-data.com)... 104.19.141.72, 104.19.142.72, 104.19.140.72, ...
Connecting to video.udacity-data.com (video.udacity-data.com)|104.19.141.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 286142532 (273M) [application/zip]
Saving to: ‘resources.zip’

resources.zip       100%[===================>] 272.89M   162MB/s    in 1.7s    

2024-07-20 08:24:08 (162 MB/s) - ‘resources.zip’ saved [286142532/286142532]



In [5]:
!unzip resources.zip

Archive:  resources.zip
   creating: resources/
  inflating: __MACOSX/._resources    
   creating: resources/video/
  inflating: __MACOSX/resources/._video  
  inflating: resources/.DS_Store     
  inflating: __MACOSX/resources/._.DS_Store  
   creating: resources/audio/
  inflating: __MACOSX/resources/._audio  
   creating: resources/text/
  inflating: __MACOSX/resources/._text  
  inflating: resources/video/Apollo11PlaqueComparison.mov  
  inflating: __MACOSX/resources/video/._Apollo11PlaqueComparison.mov  
  inflating: resources/video/Apollo11Intro.mov  
  inflating: __MACOSX/resources/video/._Apollo11Intro.mov  
  inflating: resources/video/Apollo11MoonwalkMontage.mov  
  inflating: __MACOSX/resources/video/._Apollo11MoonwalkMontage.mov  
  inflating: resources/video/OneSmallStepCompilation.mov  
  inflating: __MACOSX/resources/video/._OneSmallStepCompilation.mov  
  inflating: resources/video/RaisingTheAmericanFlag.mov  
  inflating: __MACOSX/resources/video/._RaisingTheAmericanFl


As we saw throughout this course, when working with different types of data,
we first need to parse it in a way that Gemini can understand. We will prepare our data by extracting all file names from the `resources` directory.

In [6]:
data_dir = pathlib.Path("resources/")
all_file_names = [str(file) for file in data_dir.rglob("*") if file.is_file() and not file.name.startswith('.')]

In [7]:
for file_name in all_file_names:
    print(file_name)

print(len(all_file_names))

resources/audio/Apollo11OnboardAudioHighlightClip2.mp3
resources/audio/Apollo11OnboardAudioHighlightClip4.mp3
resources/audio/Apollo11OnboardAudioHighlightClip3.mp3
resources/audio/Apollo11OnboardAudioHighlightClip5.mp3
resources/audio/Apollo11OnboardAudioHighlightClip1.mp3
resources/video/Apollo11Intro.mov
resources/video/Apollo11PlaqueComparison.mov
resources/video/OneSmallStepCompilation.mov
resources/video/BuzzDescendsCompilation.mov
resources/video/Apollo11MoonwalkMontage.mov
resources/video/RaisingTheAmericanFlag.mov
resources/text/images-020.jpg
resources/text/images-023.jpg
resources/text/images-333.jpg
14


You should expect to see 14 files.

## Retrieval Augmented Generation (RAG)

To showcase how we build a RAG, we will first build one for the Text case, and generalize it further after. Here is the general idea:
1. **Data Preparation** (done above): We first collected various types of data from NASA's public archive related to the Apollo 11 mission, including text, video, and audio files.
2. **Data Extraction and Summarization**: Extract the multimodal data from images, e.g. extract text from images using Optical Character Recognition (OCR), and use Gemini to generate summaries using a specialized prompt.
3. **Embedding Generation**: Convert the generated summaries into vector embeddings using Gemini's Text Embedding Model. These embeddings represent the summaries in a numerical format suitable for efficient similarity searches.
4. **Creating a Vector Database**: A Vector database was created to store the embeddings. This database facilitates fast and efficient retrieval of relevant documents based on similarity searches. We chose to use Chroma DB.
5. **Querying the RAG System**: For a given query, the system retrieves the most relevant documents (based on their embeddings) and generates a response using the retrieved documents as context.

Something important to note is that RAGs are usually used only when there is a surplus of data. In other words, if the data can't fit into the model prompt. In this case, the data we provided likely can fit into Gemini's 1 million token window, but for the sake of simplicity and restrictions of Google Colab's runtime, we opted to use a smaller set of data.

### Text

We will use Tesseract OCR (Optical Character Recognition) to extract text from images of the NASA report.

In [8]:
pytesseract.pytesseract.tesseract_cmd = (r'/usr/bin/tesseract')

Let's create a function to take in our images of a PDF, transcribe them into text, and summarize each of them.

In [9]:
def create_text_summary(model):
    path = pathlib.Path("resources/text")

    images = []
    text_summaries = []

    for f in path.glob("*"):
        if f.is_dir() or f.name.startswith('.'):
            continue

        try:
            # Open the image file
            image = Image.open(f)

            # Perform OCR to extract text from image
            content = pytesseract.image_to_string(image)

            # Generate the summary using the model
            response = model.generate_content(content)  # Adjust method name and arguments as per your model

            # Append the image and the generated summary to the lists
            images.append(image)
            text_summaries.append(response.text)

        except UnicodeDecodeError:
            # Handle encoding issues here, e.g., try different encodings or skip problematic files
            print(f"Error decoding {f}")
            continue

    return images, text_summaries

# Example usage:
# Initialize GenerativeModel according to its expected parameters
model = GenerativeModel()  # Adjust initialization based on your setup

In [10]:
safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]

model = genai.GenerativeModel('models/gemini-1.5-flash', safety_settings=safety_settings)

In [11]:
image_files, text_summaries = create_text_summary(model)

In [ ]:
print(dir(model))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_async_client', '_client', '_generation_config', '_get_tools_lib', '_model_name', '_prepare_request', '_safety_settings', '_system_instruction', '_tool_config', '_tools', 'cached_content', 'count_tokens', 'count_tokens_async', 'from_cached_content', 'generate_content', 'generate_content_async', 'model_name', 'start_chat']


Now, we can check out the generated summaries of the three pages we have!

In [12]:
for text_summary in text_summaries:
  print(text_summary)

This is a fascinating excerpt from the Apollo 11 Flight Plan. It provides valuable context about the document's creation and purpose, as well as the processes for managing changes. 

Here's a breakdown of the key information:

**Document Origin and Purpose:**

* **Prepared by:** Flight Planning Branch, Flight Crew Support Division
* **Technical Support:** TRW Systems
* **Purpose:** To schedule AS-506/CSM-107/LM-5 operations and crew activities to fulfill the test objectives defined in the Mission Requirements for a G Type Mission Lunar Landing.

**Timeline and Trajectory:**

* **Launch Date:** July 16, 1969
* **Launch Azimuth:** 72°
* **Trajectory:** Based on the Apollo Mission G Spacecraft Operational Trajectory provided by Mission Planning and Analysis Division

**Configuration Control and Changes:**

* **Control Board:** Crew Procedures Control Board (CPCB)
* **Change Requests:** All proposed changes that fall into the listed categories must be submitted to the CPCB via a Crew Proce

We create the Chroma database using the generated summaries. You might be wondering what Vector DB and Chroma DB are.

**Vector Database**: A specialized database designed to store and manage high-dimensional vectors, which are numerical representations of data points. It allows efficient similarity searches to find vectors (and their corresponding data) that are close to a given query vector.

**Chroma DB**: An implementation of a vector database used to store and retrieve vector embeddings. These embeddings are generated from our summaries and allow us to perform efficient similarity searches.


In [13]:
class GeminiEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        model = 'models/text-embedding-004'
        title = "Custom query"

        # Assuming `input` contains the documents or text segments to embed
        content = [doc.text for doc in input]  # Example: Extracting text from Documents object

        # Determine the appropriate task_type based on your embedding needs
        task_type = "semantic_similarity"  # Adjust based on Gemini documentation

        # Embed the content using genai library
        embedding = genai.embed_content(model=model, content=content, task_type=task_type, title=title)["embedding"]

        return Embeddings(embedding=embedding)

In [14]:
import chromadb

def create_chroma_db(documents, name):
    # Initialize ChromaDB client
    chroma_client = chromadb.Client()

    # Initialize your embedding function (GeminiEmbeddingFunction assumed here)
    embedding_function = GeminiEmbeddingFunction()

    # Create or get ChromaDB collection
    db = chroma_client.get_or_create_collection(
        name=name,
        embedding_function=embedding_function
    )

    # Add documents to the ChromaDB collection
    for i, d in enumerate(documents):
        db.add(
            document=d,
            id=str(i)
        )

    return db

In [15]:
def create_chroma_db(documents, name):
    # Initialize ChromaDB client
    chroma_client = chromadb.Client()

    # Initialize your embedding function (GeminiEmbeddingFunction assumed here)
    embedding_function = GeminiEmbeddingFunction()

    # Create or get ChromaDB collection
    db = chroma_client.get_or_create_collection(
        name=name,
        embedding_function=embedding_function
    )

    # Add documents to the ChromaDB collection
    for i, d in enumerate(documents):
        db.add(
            text=d["text"],  # Assuming each document is a dictionary with a "text" key
            id=str(i)
        )

    return db

Let's also take a peak at the `text_db` and ensure that embeddings were generated:

In [16]:
from chromadb import Client, Documents, EmbeddingFunction, Embeddings
import pandas as pd

# Define EmbeddingTaskType with required constants
class EmbeddingTaskType:
    TASK_TYPE_UNSPECIFIED = 0
    RETRIEVAL_QUERY = 1
    RETRIEVAL_DOCUMENT = 2
    SEMANTIC_SIMILARITY = 3
    CLASSIFICATION = 4
    CLUSTERING = 5
    QUESTION_ANSWERING = 6
    FACT_VERIFICATION = 7

# Define the embedding function
class GeminiEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        model = 'models/text-embedding-004'
        title = "Custom query"
        return genai.embed_content(model=model,
                                   content=input,  # Passing the input directly here
                                   task_type=EmbeddingTaskType.RETRIEVAL_DOCUMENT,  # Use the valid task type
                                   title=title)["embedding"]

# Define the function to create ChromaDB collection
def create_chroma_db(documents, name):
    chroma_client = Client()
    embedding_function = GeminiEmbeddingFunction()

    # Create or get the collection
    db = chroma_client.get_or_create_collection(name=name, embedding_function=embedding_function)

    # Add documents to the ChromaDB collection
    for i, d in enumerate(documents):
        db.add(
            documents=[d],
            ids=[str(i)]
        )

    return db

# Example documents to be added (replace with actual summaries)
documents = ["Summary 1", "Summary 2", "Summary 3"]

# Create ChromaDB collection for text summaries
text_db = create_chroma_db(documents, name="text_summaries")

# Retrieve embeddings and documents from text_db
embeddings = text_db.peek()['embeddings']
documents = text_db.peek()['documents']

# Construct a DataFrame
data = {'embeddings': embeddings, 'documents': documents}
df = pd.DataFrame.from_dict(data)

# Display the DataFrame
print(df.head())  # Displaying the first few rows for demonstration

                                          embeddings  documents
0  [-0.007464900147169828, 0.04199611023068428, -...  Summary 1
1  [-0.010615740902721882, 0.04625110328197479, -...  Summary 2
2  [0.0017052217153832316, 0.05102027207612991, -...  Summary 3


In [17]:
from google.generativeai.embedding import EmbeddingTaskType

print(EmbeddingTaskType.__members__)

{'TASK_TYPE_UNSPECIFIED': <TaskType.TASK_TYPE_UNSPECIFIED: 0>, 'RETRIEVAL_QUERY': <TaskType.RETRIEVAL_QUERY: 1>, 'RETRIEVAL_DOCUMENT': <TaskType.RETRIEVAL_DOCUMENT: 2>, 'SEMANTIC_SIMILARITY': <TaskType.SEMANTIC_SIMILARITY: 3>, 'CLASSIFICATION': <TaskType.CLASSIFICATION: 4>, 'CLUSTERING': <TaskType.CLUSTERING: 5>, 'QUESTION_ANSWERING': <TaskType.QUESTION_ANSWERING: 6>, 'FACT_VERIFICATION': <TaskType.FACT_VERIFICATION: 7>}


You should see a column called `embeddings` with what are seemingly random values, but these values are actually high-dimensional vectors that represent the semantic meaning of your summaries.

Now let's actually try querying our information. We'll test a simple example like getting some file that has to do with the Apollo 11 Flight Plan.

In [18]:
def get_relevant_files(query, db, top_k=5):
    # Perform the query to get the top K relevant results
    results = db.query(
        query_texts=[query],
        n_results=top_k
    )

    # Print the results to understand their structure
    print(results)

    # Extract the documents directly from the query results
    relevant_files = results['documents'][0]  # The documents are in a nested list

    return relevant_files

# Example usage:
query = "Sample query text"
relevant_files = get_relevant_files(query, text_db, top_k=5)

# Print out the relevant files
print(relevant_files)

{'ids': [['1', '0', '2']], 'distances': [[0.5705089569091797, 0.5824756026268005, 0.5931833386421204]], 'metadatas': [[None, None, None]], 'embeddings': None, 'documents': [['Summary 2', 'Summary 1', 'Summary 3']], 'uris': None, 'data': None, 'included': ['metadatas', 'documents', 'distances']}
['Summary 2', 'Summary 1', 'Summary 3']


In [19]:
files = get_relevant_files("Apollo 11 Flight Plan", text_db)
print(files)

{'ids': [['0', '1', '2']], 'distances': [[0.9854038953781128, 0.9945674538612366, 1.003105878829956]], 'metadatas': [[None, None, None]], 'embeddings': None, 'documents': [['Summary 1', 'Summary 2', 'Summary 3']], 'uris': None, 'data': None, 'included': ['metadatas', 'documents', 'distances']}
['Summary 1', 'Summary 2', 'Summary 3']


You should expect to see something like `['1', '0', '2']`. This means that the first entry in the `text_db` is most similar. If you look above at our `pd.DataFrame` output, the document with id 1 is the document about the Apollo 11 Flight Plan, so this is working as we expected!

### Video and Audio

Congrats! You've successfully built a working RAG for text. Now, let's extend this concept to Video and Audio, and build out some more complex queries. We'll begin by generalizing the above summary creation function to all sorts of modalities.

In [20]:
def create_summary(modality):
  path = data_dir / modality

  summary_prompt = f"""You are an assistant tailored for summarizing {modality} for retrieval.
  These summaries will be turned into vector embeddings and used to retrieve the raw {modality}.
  Give a concise summary of the {modality} that is well optimized for retrieval. Here is the {modality}."""

  files = []
  summaries = []

  for f in path.glob("*"):
    if f.is_dir() or f.name.startswith('.'):
      continue
    print(f)

    if modality == "text":
      file = Image.open(f)
      response = model.generate_content([summary_prompt, pytesseract.image_to_string(file)])

    else:
      file = genai.upload_file(f)

      while file.state.name == "PROCESSING":
        print("Waiting for video file upload...\n", end='')
        time.sleep(5)
        file = genai.get_file(file.name)

      response = model.generate_content([summary_prompt, file])

    files.append(file)
    summaries.append(response.text)

  return files, summaries

In [ ]:
!pip list

Package                                  Version
---------------------------------------- ---------------------
absl-py                                  1.4.0
aiohttp                                  3.9.5
aiosignal                                1.3.1
alabaster                                0.7.16
albumentations                           1.3.1
altair                                   4.2.2
annotated-types                          0.7.0
anyio                                    3.7.1
argon2-cffi                              23.1.0
argon2-cffi-bindings                     21.2.0
array_record                             0.5.1
arviz                                    0.15.1
asgiref                                  3.8.1
astropy                                  5.3.4
astunparse                               1.6.3
async-timeout                            4.0.3
atpublic                                 4.1.0
attrs                                    23.2.0
audioread                            

Now, we will create a folder with all of our data of different modalities. In particular, the first 5 are audio files, next 3 are text files, and final 6 are video files.

In [25]:
all_files = []
all_summaries = []

for modality_type in ["audio", "text", "video"]:
  files, summaries = create_summary(modality_type)
  all_files.extend(files)
  all_summaries.extend(summaries)

resources/audio/Apollo11OnboardAudioHighlightClip2.mp3
resources/audio/Apollo11OnboardAudioHighlightClip4.mp3
resources/audio/Apollo11OnboardAudioHighlightClip3.mp3
resources/audio/Apollo11OnboardAudioHighlightClip5.mp3
resources/audio/Apollo11OnboardAudioHighlightClip1.mp3
resources/text/images-020.jpg
resources/text/images-023.jpg
resources/text/images-333.jpg
resources/video/Apollo11Intro.mov
Waiting for video file upload...
Waiting for video file upload...
resources/video/Apollo11PlaqueComparison.mov
Waiting for video file upload...
resources/video/OneSmallStepCompilation.mov
Waiting for video file upload...
resources/video/BuzzDescendsCompilation.mov
Waiting for video file upload...
resources/video/Apollo11MoonwalkMontage.mov
Waiting for video file upload...
resources/video/RaisingTheAmericanFlag.mov
Waiting for video file upload...


In [26]:
db = create_chroma_db(all_summaries, "nasa")

Again, ensure that the embeddings were generated. Notice that now, we have audio, video, and text data.

In [27]:
data = {
    'embeddings': db.peek()['embeddings'],
    'documents': db.peek()['documents']
}

df = pd.DataFrame.from_dict(data, orient='index').transpose()
df

,embeddings,documents
0,"[0.06868059933185577, -0.02138531021773815, -0...",The audio clip is a conversation about a space...
1,"[0.005218075588345528, 0.007501657120883465, 0...",Two people are flying a plane and discussing t...
2,"[0.025413116440176964, -0.04366571828722954, -...",This video shows a side-by-side comparison of ...
3,"[0.04278874397277832, -0.041772522032260895, -...",Buzz Aldrin descends the Apollo 11 Lunar Modul...
4,"[0.0379268154501915, -0.005956900306046009, -0...",Astronauts talk about the surface of the moon ...
5,"[0.03411030396819115, -0.020206548273563385, -...",This video compares the NASA archive footage o...
6,"[0.045547883957624435, -0.03362090513110161, 0...",A group of astronauts discuss the upcoming lun...
7,"[-0.02427656017243862, 0.013114254921674728, -...",This audio is a recording of two people having...
8,"[0.014638802036643028, -0.014723807573318481, ...",Two people are talking about a plane flight. T...
9,"[0.07213490456342697, 0.023696381598711014, 0....",This document outlines the Apollo 11 mission p...


In [28]:
files = get_relevant_files("communication with Mission Control", db)
print(files)

{'ids': [['0', '5', '12', '4', '1']], 'distances': [[0.6343160271644592, 0.8111491203308105, 0.8211809396743774, 0.8587968945503235, 0.8711273074150085]], 'metadatas': [[None, None, None, None, None]], 'embeddings': None, 'documents': [['The audio clip is a conversation about a space mission. The crew members are discussing the various readings and parameters of the mission, including chamber pressure, trim values, and rate values. The discussion focuses on ensuring the proper functioning of the spacecraft systems and addressing any discrepancies or anomalies. \n', 'This document outlines the Apollo 11 mission plan, detailing the operations and crew activities for the AS-506/CSM-107/LM-5 spacecraft. The plan is based on a July 16, 1969 launch with a 72° azimuth and aims to fulfill the test objectives defined in the Mission Requirements for a G Type Mission Lunar Landing. The document is under the control of the Crew Procedures Control Board (CPCB) and any proposed changes must be submi

Can we do more than just return the most relevant file? Yes we can! We can ask Gemini to return a response to the query using the files it thinks are most relevant, provide an answer and tell us what files it used! This is really exciting, and has vast applications in many industries.

In [35]:
def query_rag(query, db):
    files = get_relevant_files(query, db)
    # Ensure all_files is a dictionary mapping file identifiers to content
    all_files = {f: "content for file " + f for f in files} # Replace with actual content retrieval
    prompt = [all_files.get(f, "") for f in files]
    prompt.append("Generate a response to the query using the provided files. Here is the query.")
    prompt.append(query)
    # Ensure all_file_names is a dictionary
    all_file_names = {f: "name for file " + f for f in files} # Replace with actual name retrieval
    return model.generate_content(prompt).text, [all_file_names.get(f, "") for f in files]

In [36]:
for response in query_rag("Explain what happened with the Apollo 11 Mission.", db):
    print(response)

{'ids': [['8', '10', '9', '6', '5']], 'distances': [[0.5989505052566528, 0.6398935317993164, 0.6904922723770142, 0.6948361396789551, 0.7009021043777466]], 'metadatas': [[None, None, None, None, None]], 'embeddings': None, 'documents': [['Footage of the Apollo 11 mission including the launch, landing, and splashdown. Neil Armstrong walks on the moon and utters his famous quote, "That\'s one small step for man, one giant leap for mankind."  Footage of the lunar surface and the astronauts exploring it. \n', 'This video shows a side-by-side comparison of footage from the Apollo 11 mission. The left side shows the original footage from NASA’s archive, and the right side shows restored footage from 2009. The video focuses on Neil Armstrong’s descent down the lunar module ladder to the surface of the moon. ', 'A side-by-side comparison of an original and restored image of the Apollo 11 lunar module landing. Buzz Aldrin and Neil Armstrong are shown on the moon revealing the plaque that states 

In [37]:
for response in query_rag("What happens at the Translunar Coast in the Mission Description?", db):
    print(response)

{'ids': [['6', '5', '0', '12', '2']], 'distances': [[0.7484702467918396, 0.8618978261947632, 0.8928728103637695, 0.9176099300384521, 0.9339534640312195]], 'metadatas': [[None, None, None, None, None]], 'embeddings': None, 'documents': [['This document outlines the mission plan for the Apollo 11 lunar landing, detailing the launch, Earth orbit insertion, Translunar Coast (TLC), and lunar orbit insertion (LOI) phases. Key events during TLC include: transposition, docking, and LM ejection, separation from the SIVB and an evasive maneuver, SIVB propulsive venting, navigation sightings, and midcourse corrections. The document also provides specific timings for these events and references a table for detailed burn data. \n', 'This document outlines the Apollo 11 mission plan, detailing the operations and crew activities for the AS-506/CSM-107/LM-5 spacecraft. The plan is based on a July 16, 1969 launch with a 72° azimuth and aims to fulfill the test objectives defined in the Mission Requirem

In [38]:
for response in query_rag("REPLACE ME: Ask any questions you'd like here about Apollo 11!", db):
    print(response)

{'ids': [['10', '8', '9', '11', '12']], 'distances': [[0.6383141875267029, 0.6410767436027527, 0.7120367288589478, 0.7285556197166443, 0.7324788570404053]], 'metadatas': [[None, None, None, None, None]], 'embeddings': None, 'documents': [['This video shows a side-by-side comparison of footage from the Apollo 11 mission. The left side shows the original footage from NASA’s archive, and the right side shows restored footage from 2009. The video focuses on Neil Armstrong’s descent down the lunar module ladder to the surface of the moon. ', 'Footage of the Apollo 11 mission including the launch, landing, and splashdown. Neil Armstrong walks on the moon and utters his famous quote, "That\'s one small step for man, one giant leap for mankind."  Footage of the lunar surface and the astronauts exploring it. \n', 'A side-by-side comparison of an original and restored image of the Apollo 11 lunar module landing. Buzz Aldrin and Neil Armstrong are shown on the moon revealing the plaque that state

In [ ]:
import anvil.server

anvil.server.connect("server_CL25YXN533KRSW7GVR5TCG4Y-GF34G3VBV7X2WH32")

Congrats! You've built a full end to end multimodal RAG with just a few tools. We hope you enjoyed following along in this notebook and learned a lot on the way.